In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dropout
from keras.callbacks import EarlyStopping

2024-08-26 14:48:38.009178: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-26 14:48:38.011532: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-26 14:48:38.019005: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-26 14:48:38.030818: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-26 14:48:38.034493: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-26 14:48:38.043605: I tensorflow/core/platform/cpu_feature_gu

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [4]:
# Loading dataset
df = pd.read_csv('../clean_data/sarcasm_headlines_v2_clean.csv')

In [5]:
# Checking if dataset is correctly loaded
df.head()

,is_sarcastic,cleaned_headline
0,1,thirtysomething scientist unveil doomsday cloc...
1,0,dem rep totally nail why congress be fall shor...
2,0,eat your veggie deliciously different recipe
3,1,inclement weather prevent liar from get to work
4,1,mother come pretty close to use word stream co...


In [6]:
df.shape

(28619, 2)

In [7]:
# Checking max length
len(df['cleaned_headline'].max())

92

In [8]:
# Model parms
max_len = 100
embedding_dim = 200
max_features = 5000
num_filters = 32

# Model fit params
batch_size = 128
num_epochs = 8

In [9]:
# Define X and y
X = df['cleaned_headline']
y = df['is_sarcastic']

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Tokenizing
tfid_vectorizer = TfidfVectorizer(max_df = 0.75, max_features = max_features, ngram_range=(1,2))
X_processed = pd.DataFrame(tfid_vectorizer.fit_transform(df['cleaned_headline']).toarray(),
                              columns=tfid_vectorizer.get_feature_names_out())

In [11]:
# Splitting data
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=.3, random_state=42, shuffle=True)

In [12]:
# Checking shape
X_train.shape

(20033, 5000)

In [13]:
# Instanciating model
model = Sequential([
    Embedding(max_features, embedding_dim, trainable=False),
    Conv1D(num_filters, 7, padding='same', activation='relu'),
    MaxPooling1D(pool_size=2),
    Dropout(0.5),
    LSTM(64, dropout=0.5, recurrent_dropout=0.3),
    Dense(1, activation='sigmoid')
])

I0000 00:00:1724676521.239029   78339 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-26 14:48:41.239339: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [14]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Defining early stopping params
es = EarlyStopping(
    monitor = 'val_loss',
    patience = 5,
    verbose = 1,
    restore_best_weights = True
)

In [15]:
# Train the model
history = model.fit(X_train, y_train, epochs=num_epochs, validation_data=(X_test, y_test), callbacks=[es])

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test loss: {loss:.4f}')
print(f'Test accuracy: {accuracy:.4f}')

Epoch 1/8
 13/627 ━━━━━━━━━━━━━━━━━━━━ 9:19 911ms/step - accuracy: 0.5302 - loss: 0.6940

KeyboardInterrupt: 